In [77]:
import pandas as pd
import numpy as np
import os

In [78]:
import os

def count_files_in_folder(folder_path):
    file_count = 0

    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isfile(full_path):
            file_count += 1
    
    return file_count

print(f"Files in folder: {count_files_in_folder("data/")}.")

Files in folder: 349.


# Notes
WSZYSTKIE PLIKI SĄ IDENTYCZNE. WSZYSTKIE TO EXCEL - 14 kart
Czyszczenie:
•	Wszystkie NaN na 0
•	Skalowanie przed modelem
•	Inżynieria cech (wskaźniki ekonomiczne, różnica miedzy każym z elementów)

TESTY DO PRZEPROWADZENIA:
•	Uwzglednienie wszystkiego
•	Skupienie sie na zmianach kapitałowych
•	Ograniczenie tylko główne wartości
•	Grupowanie po przedsiębiorstwie? 

Podsumowanie co wyciągnąć z pliku:
Dodatkowa kolumna jako nazwa pliku
C – nazwa kolumny w DF

(Kolumny: OD M DO AB. Wiersze: Od 3 do 18 włącznie)
(wiersze 30 do 93 włącznie)
Wiersze 255 – 275 włącznie
Wiersze 279 – 287 włącznie (dane w %)
Wiersze 290 – 293 włącznie

In [79]:
folder_path = 'dataa/'

In [80]:
if os.path.exists(folder_path):
    print("Folder exists. Proceeding with file processing.")
else:
    print("Folder does not exist. Check the path.")

Folder exists. Proceeding with file processing.


In [81]:
# TEST, przejście po każdym pliku i wyciągnięcie z 'Info' nazwa firmy i sektora

In [82]:
def test_process_excel_files(folder_path):
    results = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                with pd.ExcelFile(file_path) as xls:
                    company_name = pd.read_excel(xls, 'Info', usecols="B", skiprows=1, nrows=1).values[0][0]
                    sector = pd.read_excel(xls, 'Info', usecols="E", skiprows=19, nrows=1).values[0][0]
                
                result = {
                    'filename': filename,
                    'Company Name': company_name,
                    'Sector': sector
                }
                results.append(result)
                
                print(f"Dane z {filename} zostały przetworzone.")
            except Exception as e:
                print(f"Błąd przy przetwarzaniu pliku {filename}: {e}")

    results_df = pd.DataFrame(results)
    return results_df

In [83]:
test_process_excel_files(folder_path)

Dane z ZYWIEC.xlsx zostały przetworzone.


,filename,Company Name,Sector
0,ZYWIEC.xlsx,Grupa Żywiec SA,napoje


# Przygotowanie DF z danymi (kolumny)

In [84]:
def process_excel_data(base_folder):
    folder_path = os.path.join(base_folder)
    all_data = []

    row_ranges = [
        range(2, 19),  # Rows 3 to 18 inclusive, zero-indexed
        range(29, 94),  # Rows 30 to 93 inclusive
        range(254, 276),  # Rows 255 to 275 inclusive
        range(278, 288),  # Rows 279 to 287 inclusive
        range(289, 294)  # Rows 290 to 293 inclusive
    ]

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                data = pd.read_excel(file_path, sheet_name='QS', usecols=[2], header=None)

                extracted_data = []
                for row_range in row_ranges:
                    range_data = data.iloc[row_range].squeeze().tolist()
                    concatenated_data = ', '.join(map(str, range_data))
                    extracted_data.append(concatenated_data)

                new_row = {}
                for item in extracted_data:
                    for entry in item.split(', '):
                        new_row[entry.strip()] = None  # dummy value

                new_df = pd.DataFrame([new_row])
                new_df['filename'] = filename

                all_data.append(new_df)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    results_df = pd.concat(all_data, ignore_index=True)
    return results_df

df_results = process_excel_data(folder_path)
df_results

,End of period,Start of period,Revenues from sales,Operating profit/loss,Net profit/loss attributable to equity holders of the parent,Total comprehensive income attributable to equity holders,Depreciation,Cash flow from operating activities,Cash flow from investing activities,Cash flow from financing activities,...,Leverage (EM),Asset utilization (AU),Load gross profit,Load operating profit,EBITDA margin,EBITDA,Current financial liabilities,Non-current financial liabilities,Financial liabilities (total),filename
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,ZYWIEC.xlsx


In [90]:
import os
import pandas as pd
import numpy as np

def process_excel_data(base_folder):
    folder_path = os.path.join(base_folder)
    all_data = []

    row_ranges = [
        range(2, 19),  # Rows 3 to 18 inclusive, zero-indexed
        range(29, 94),  # Rows 30 to 93 inclusive
        range(254, 276),  # Rows 255 to 275 inclusive
        range(278, 288),  # Rows 279 to 287 inclusive
        range(289, 294)  # Rows 290 to 293 inclusive
    ]

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                data = pd.read_excel(file_path, sheet_name='QS', usecols="D:CX", header=None)
                
                headers = data.iloc[2, :].tolist()

                data = data.replace(0, np.nan)
                
                extracted_data = []
                for row_range in row_ranges:
                    for row in row_range:
                        row_data = data.iloc[row, :].tolist()
                        row_dict = {headers[i]: row_data[i] for i in range(len(headers))}
                        row_dict['filename'] = filename
                        extracted_data.append(row_dict)

                all_data.extend(extracted_data)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    results_df = pd.DataFrame(all_data)
    return results_df

df_results = process_excel_data(folder_path)
print(df_results)


     1998-03-31  1998-06-30  1998-09-30  1998-12-31  1999-03-31  1999-06-30  \
0    1998-03-31  1998-06-30  1998-09-30  1998-12-31  1999-03-31  1999-06-30   
1    1998-01-01  1998-04-01  1998-07-01  1998-10-01  1999-01-01  1999-04-01   
2        126003      142773      222098      154110      168924      271113   
3         11171       50830       61141       27972       31190       62798   
4          5812       33203       41145       18590       18760       42065   
..          ...         ...         ...         ...         ...         ...   
114       22613       62839       75136       40521       43577       82357   
115         NaN         NaN         NaN         NaN         NaN         NaN   
116         NaN         NaN         NaN         NaN         NaN         NaN   
117         NaN         NaN         NaN         NaN         NaN         NaN   
118         NaN         NaN         NaN         NaN         NaN         NaN   

     1999-09-30  1999-12-31  2000-03-31  2000-06-30